In [1]:
import skimage.feature, skimage.io
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

#Pasta que contém o conjunto de dados já extraído.
#Esta pasta contém as pastas A-Z e os arquivos NIST_Train_Upper.txt, NIST_Test_Upper.txt e NIST_Valid_Upper.txt
NCharacter_dataset_folder = "../exercicios/NCharacter_SD19_BMP/"

#Uma pasta onde as características extraídas são salvas.
features_folder = 'features'

#Classe que abstrai a divisão em zonas
class Zonas(object):
    def __init__(self):
        pass
    
    #Função geradora que retorna uma zona de cada vez.
    #Os argumentos são o número de linhas da imagem e o número de colunas da imagem.
    def get_zonas(self, n_linhas, n_colunas):
        raise NotImplementedError

#Classe que implementa a divisão em zonas retangulares.
class ZonasRetangulares(Zonas):
    
    #Construtor que configura o zoneamento a ser feito: zonas_x zonas horizontais e zonas_y zonas verticais.
    def __init__(self, zonas_x, zonas_y):
        super(ZonasRetangulares, self).__init__()
        self.zonas_x = zonas_x
        self.zonas_y = zonas_y
        
    #Implementa a função geradora de zonas retangulares.
    def get_zonas(self, n_linhas, n_colunas):
        cortes_x = np.floor(np.linspace(0, n_colunas, num=self.zonas_x+1)).astype(int)
        cortes_y = np.floor(np.linspace(0, n_linhas, num=self.zonas_y+1)).astype(int)
        #print (cortes_x)
        #print (cortes_y)
        for i in range(len(cortes_x)-1):
            for j in range(len(cortes_y)-1):
                yield(cortes_x[i], cortes_y[j], cortes_x[i+1], cortes_y[j+1],cortes_x[i] - cortes_x[i+1], cortes_y[i] - cortes_y[i+1] )


# Caraterísticas

In [2]:
def histograma_cor(imagem):
    #print('im', imagem.shape)
    vals, counts = np.unique(imagem, return_counts=True)
    o = vals.argsort()
    vals = vals[o]
    counts = counts[o]
    if len(vals) < 2:
        #imagem com apenas branco ou apenas preto
        if vals[0] == 0:
            return [counts[0], 0]
        else:
            return [0, counts[0]]
        
    return counts

# Extração de Características

In [3]:
#Esta função apenas abre os arquivos com as listas de treino / teste e validação.
#Retorna os caminhos para os arquivos, os rótulos e o nome dos arquivos sem o caminho.
def parse_filelist(path, prefix=''):
    with open(path, 'r') as f:
        c = f.readlines()
    caminhos = list(map(str.strip, c))
    rotulos = [ i.split('/')[1].upper() for i in caminhos]
    arquivos = [i.split('/')[-1] for i in caminhos]
    p = zip(rotulos, arquivos)
    caminhos = [ prefix + '/' + i[0] + '/' + i[1] for i in p]
    
    return list(zip(caminhos,rotulos, arquivos))

#Esta é uma função que recebe o tipo de zoneamento a ser feito e as features que devem ser
#extraídas de cada zona. Veja que essa função é chamada no "for" abaixo, que faz a extração das características
#para cada uma das listas de imagens (treino, teste e validação)
def extract_features(filelist, dataset_folder, zonas, features=[histograma_cor]):
    instancias = parse_filelist(filelist, prefix=dataset_folder)
    #Note que o MAP abaixo mapeia cada instância (imagem) à função que extrai as 
    #características (feature_extraction) abaixo.
    features = list(map(feature_extraction, instancias, [zonas] * len(instancias), [features] * len(instancias)))
    
    return np.array(features)

#Essa função de extração das características de cada instância. zonas é uma instância de subclasse
#de Zonas. Features é uma lista de funções que extraem características. Cada função da lista recebe uma matriz
#que representa a imagem (que está na zona) e retorna o vetor de característica computado daquela característica. 
def feature_extraction(instancia, zonas, features):
    caminho = instancia[0]
    #print(instancia)
    imagem = skimage.io.imread(caminho)
    caracteristicas = np.array([])
    #print("imagem.shape",imagem.shape)
    
    res = []
    for f in features:
        for z in zonas.get_zonas(imagem.shape[1], imagem.shape[0]):
            #print ("%d:%d,%d:%d" % (z[0], z[2], z[1], z[3]))
            f_val = f(imagem[z[0]:z[2],z[1]:z[3]])
            res.extend(f_val)
    
    return np.array(res)

#Realiza a extração das características!
for i in [('NIST_Train_Upper.txt', 'train'), ('NIST_Test_Upper.txt', 'test'), ('NIST_Valid_Upper.txt', 'val')]:
    print('extraindo características de %s' % (i[0]))
    #note que esta linha extrai características de 4 zonas (2 zonas por linha e 2 por coluna). 
    #Neste exemplo apenas a característica histograma_cor é computada.
    feats = extract_features(NCharacter_dataset_folder + i[0], NCharacter_dataset_folder, ZonasRetangulares(2,2), features=[histograma_cor])
    
    #Extraia aqui outras características! Escolha outros zoneamentos! 
    #Exemplo: feats2 recebe as características de histograma de cor calculadas de 9 zonas 
    feats2 = extract_features(NCharacter_dataset_folder + i[0], NCharacter_dataset_folder, ZonasRetangulares(3,3), features=[histograma_cor])
    
    
    #Concatena feats e feats 2 em feats.
    feats = np.concatenate((feats,feats2),axis=1)
    
    #np.save(open(features_folder + ('/%s_feats.pkl' % (i[1])), 'wb'), feats )
    np.save(features_folder + ('/%s_feats.pkl' % (i[1])), feats)

print('Fim da extração de características!')



extraindo características de NIST_Train_Upper.txt
extraindo características de NIST_Test_Upper.txt
extraindo características de NIST_Valid_Upper.txt
Fim da extração de características!


# Treino e Teste com SVM e KNN (Sem validação)

In [6]:
train_features = np.load(features_folder + '/train_feats.pkl.npy')
train_rotulos = parse_filelist(NCharacter_dataset_folder + 'NIST_Train_Upper.txt')
train_rotulos = [i[1] for i in train_rotulos]
print (len(train_rotulos), train_features.shape)

test_features = np.load(features_folder + '/test_feats.pkl.npy')
test_rotulos = parse_filelist(NCharacter_dataset_folder + 'NIST_Test_Upper.txt')
test_rotulos = [i[1] for i in test_rotulos]
print (len(test_rotulos), test_features.shape)

SS = StandardScaler()
SS.fit(train_features)
train_features = SS.transform(train_features)
test_features = SS.transform(test_features)

KNN = KNNC(n_neighbors=3)

KNN.fit(train_features, train_rotulos)


y_pred = KNN.predict(test_features)

print('KNN accuracy score: ' , '{:.3f}'.format(accuracy_score(test_rotulos, y_pred)))
#print(confusion_matrix(test_rotulos, y_pred))

print(classification_report(test_rotulos, y_pred, digits=3))

clf = SVC()
clf.fit(train_features, train_rotulos)
y_pred = clf.predict(test_features)

print('SVM accuracy score: ', '{:.3f}'.format(accuracy_score(test_rotulos, y_pred)))
#print(confusion_matrix(test_rotulos, y_pred))

print(classification_report(test_rotulos, y_pred, digits=3))

37440 (37440, 26)
11941 (11941, 26)


/usr/local/lib64/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


KNN accuracy score:  0.691
             precision    recall  f1-score   support

          A      0.661     0.791     0.720       459
          B      0.448     0.605     0.515       435
          C      0.717     0.857     0.781       518
          D      0.460     0.795     0.583       396
          E      0.486     0.614     0.542       365
          F      0.663     0.704     0.683       419
          G      0.518     0.599     0.555       389
          H      0.601     0.799     0.686       402
          I      0.958     0.697     0.807       815
          J      0.746     0.779     0.762       426
          K      0.523     0.629     0.571       377
          L      0.864     0.964     0.911       496
          M      0.492     0.341     0.403       460
          N      0.660     0.569     0.611       439
          O      0.752     0.436     0.552       459
          P      0.760     0.752     0.756       467
          Q      0.680     0.540     0.602       452
          R      0